In [1]:
import pandas as pd

import os

import datetime

from api.api_facebook import get_facebook
from api.api_googleAds import get_googleAds
from api.api_campaign_manager import get_CampaignManager
from api.api_twitter import get_twitter
from api.api_tiktok import get_tiktok

from api.api_googleAnalytics import get_googleAnalytics_bm
from api.api_googleAnalytics import get_googleAnalytics_cm

from functions import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
campaigns = pd.read_excel(os.path.abspath(os.path.join('..', 'campanhas', 'campanhas.xlsx')))
campaigns.fillna('', inplace = True)

In [4]:
campaign_dir = os.path.abspath(os.path.join('..', 'campanhas'))
paramet_dir = os.path.abspath(os.path.join('..', 'parametrização'))

campaigns = pd.read_excel(os.path.abspath(os.path.join(campaign_dir, 'campanhas.xlsx')))
campaigns.fillna('', inplace = True)

In [10]:
list_campaings = []

for _, campaign in campaigns.iterrows():

    is_active = False
    is_updated = False
    have_bm = False
    have_cm = False

    is_facebook = False
    is_google = False
    is_twitter = False
    is_tiktok = False
    is_analytics = False

    campaign_name = campaign['campanha']
    campaign_path = os.path.abspath(os.path.join(campaign_dir, campaign_name.lower()))

    campaign_param_path = os.path.abspath(os.path.join(paramet_dir, campaign['nome_parametrizacao'] + '.xlsm'))

    if not os.path.isfile(campaign_param_path):
        campaign_param_path = os.path.abspath(os.path.join(paramet_dir, campaign['nome_parametrizacao'] + '.xlsx'))

    campaign_start = campaign['data_inicio'].date()
    campaign_end = campaign['data_fim'].date()

    today = datetime.datetime.today().date()
    yesterday = today - datetime.timedelta(days = 1)

    init_date = campaign_start
    end_date = yesterday

    if campaign_start < today <= campaign_end + datetime.timedelta(days = 1):
        is_active = True

    if not os.path.isdir(campaign_path):
        os.mkdir(campaign_path)
        os.mkdir(os.path.join(campaign_path, 'data'))
        os.mkdir(os.path.join(campaign_path, 'source'))
        
    if is_active:
        param = pd.read_excel(campaign_param_path, sheet_name = 'mas_parametrizacao')
        param.drop(columns = ['id_parametro.1', 'term'], inplace = True)
        param.fillna('', inplace = True)

        param.rename(columns = {'Nome do criativo (para programação nas plataformas)': 'ad_name'}, inplace = True)

        facebook_accounts = campaign['facebook'].split(',') if campaign['facebook'] else False
        google_accounts = campaign['google'].split(',') if campaign['google'] else False
        tiktok_accounts = campaign['tiktok'].split(',') if campaign['tiktok'] else False
        twitter_campaigns = campaign['twitter'].split(',') if campaign['twitter'] else False
        cm_campaigns = campaign['campaign_manager'].split(',') if campaign['campaign_manager'] else False

        view_id = str(campaign['ga'])

        if view_id:
            is_analytics = True

        print(f'Atualizando campanha: {campaign_name}')

        source_folder = os.path.abspath(os.path.join(campaign_path, 'source'))
        pr = os.path.abspath(os.path.join(source_folder, 'concat_pr.xlsx'))
        gaov = os.path.abspath(os.path.join(source_folder, 'concat_ga.xlsx'))

        if os.path.isfile(gaov):
            temp_gaov = pd.read_excel(gaov, sheet_name = 'concat_gaov')

            if temp_gaov.date.max() == yesterday:
                is_updated = True
                print(f'{campaign_name} já atualizado!')

        concat_pr = []
        concat_ov = []
        concat_ev = []

        if not is_updated:
            if facebook_accounts:
                have_bm = True
                is_facebook = True

                facebook = update_bm(campaign_path, init_date, bm = 'Facebook', accounts = facebook_accounts, bm_function = get_facebook)
                facebook.fillna('', inplace = True)

                no_content_facebook = facebook[facebook['content'] == '']
                content_facebook = facebook.drop(facebook[facebook['content'] == ''].index)

                content_facebook_pr = merge_params(content_facebook, param, campaign_path, 'facebook', content = True)
                no_content_facebook_pr = merge_params(no_content_facebook, param, campaign_path, 'facebook', content = False)

                facebook_pr = pd.concat([content_facebook_pr, no_content_facebook_pr], axis = 0, ignore_index = True)
                facebook_pr.sort_values('date', inplace = True, ignore_index = True)
                concat_pr.append(facebook_pr)

            if google_accounts:
                have_bm = True
                is_google = True

                googleAds = update_bm(campaign_path, init_date, bm = 'GoogleAds', accounts = google_accounts, bm_function = get_googleAds)
                googleAds.fillna('', inplace = True)

                no_content_googleAds = googleAds[googleAds['content'] == '']
                content_googleAds = googleAds.drop(googleAds[googleAds['content'] == ''].index)  

                content_googleAds_pr = merge_params(content_googleAds, param, campaign_path, 'googleAds', content = True)
                no_content_googleAds_pr = merge_params(no_content_googleAds, param, campaign_path, 'googleAds', content = False)

                googleAds_pr = pd.concat([content_googleAds_pr, no_content_googleAds_pr], axis = 0, ignore_index = True)
                googleAds_pr.sort_values('date', inplace = True, ignore_index = True)
                concat_pr.append(googleAds_pr)

            if tiktok_accounts:
                have_bm = True
                is_tiktok = True

                # tiktok = update_bm(campaign_path, init_date, bm = 'TikTok', accounts = tiktok_accounts, bm_function = get_tiktok)
                # tiktok.fillna('', inplace = True)

                tiktok = pd.read_excel('V:\\BANCO_DO_BRASIL\\2022\\_INSTITUCIONAL\\DADOS_BI\\BASE_DE_DADOS\\campanhas\\oportunidades\\data\\tiktok_prep.xlsx', sheet_name = f'tiktok_prep', parse_dates = ['date'])
                tiktok_pr = merge_params(tiktok, param, campaign_path, 'TikTok', content = False)

                tiktok_pr.sort_values('date', inplace = True, ignore_index = True)
                concat_pr.append(tiktok_pr)

            if twitter_campaigns:
                have_bm = True
                is_twitter = True

                twitter = update_bm(campaign_path, init_date, bm = 'Twitter', campaigns = twitter_campaigns, bm_function = get_twitter)
                twitter.fillna('', inplace = True)

                no_content_twitter = twitter[twitter['content'] == '']
                content_twitter = twitter.drop(twitter[twitter['content'] == ''].index)   

                content_twitter_pr = merge_params(content_twitter, param, campaign_path, 'twitter', content = True)
                no_content_twitter_pr = merge_params(no_content_twitter, param, campaign_path, 'twitter', content = False)
                        
                twitter_pr = pd.concat([content_twitter_pr, no_content_twitter_pr], axis = 0, ignore_index = True)
                twitter_pr.sort_values('date', inplace = True, ignore_index = True)
                concat_pr.append(twitter_pr)

            if cm_campaigns:
                have_cm = True

                campaign_manager = update_bm(campaign_path, init_date, bm = 'CampaignManager', campaigns = cm_campaigns, bm_function = get_CampaignManager)
                campaign_manager_pr = merge_cm_params(campaign_manager, param, campaign_path)

                campaign_manager_pr.sort_values('date', inplace = True, ignore_index = True)
                concat_pr.append(campaign_manager_pr)

            print('Criando base de BM.')
            concat_pr_all = pd.concat(concat_pr, axis = 0, ignore_index = True)
            concat_pr_all.to_excel(pr, index = False, sheet_name = 'concat_pr')

            if have_bm and is_analytics:
                bm_campaigns = []
                bm_campaigns += list(googleAds.campaign.str.replace('  ', ' ').unique())                        
                bm_campaigns += list(facebook.campaign.unique())

                bm_overview, bm_events = update_ga(campaign_path, view_id, init_date, ga = 'bm', campaigns = bm_campaigns, ga_function = get_googleAnalytics_bm)

                if is_facebook:
                    facebook_ov, facebook_ev = merge_ga(facebook_pr, bm_overview, bm_events, bm = 'facebook')
                    concat_ov.append(facebook_ov)
                    concat_ev.append(facebook_ev)
                        
                if is_google:
                    googleAds_ov, googleAds_ev = merge_ga(googleAds_pr, bm_overview, bm_events, bm = 'google')
                    concat_ov.append(googleAds_ov)
                    concat_ev.append(googleAds_ev)

                # if is_tiktok:
                #     tiktok_ov, tiktok_ev = merge_ga(tiktok_pr, bm_overview, bm_events, bm = 'tiktok')
                #     concat_ov.append(tiktok_ov)
                #     concat_ev.append(tiktok_ev)

                if is_twitter:
                    twitter_ov, twitter_ev = merge_ga(twitter_pr, bm_overview, bm_events, bm = 'twitter')
                    concat_ov.append(twitter_ov)
                    concat_ev.append(twitter_ev)

            if have_cm and is_analytics:
                cm_overview, cm_events = update_ga(campaign_path, view_id, init_date, ga = 'cm', campaigns = cm_campaigns, ga_function = get_googleAnalytics_cm)
                campaign_manager_ov, campaign_manager_ev = merge_ga_cm(campaign_manager_pr, cm_overview, cm_events)
                concat_ov.append(campaign_manager_ov)
                concat_ev.append(campaign_manager_ev)
                            
            if is_analytics:
                concat_gaov = pd.concat(concat_ov, axis = 0, ignore_index = True)                    
                concat_gaev = pd.concat(concat_ev, axis = 0, ignore_index = True)

                print('Criando base de GA.')
                writer = pd.ExcelWriter(gaov)
                concat_gaov.to_excel(writer, index = False, sheet_name = 'concat_gaov')
                concat_gaev.to_excel(writer, index = False, sheet_name = 'concat_gaev')
                writer.save()

            if campaign_name == 'Soluções Digitais':
                adjust = update_adjust(campaign_path, init_date)

                adjust_os = os.path.abspath(os.path.join(source_folder, 'adjust_prep.xlsx'))
                adjust.to_excel(adjust_os, index = False, sheet_name = 'adjust_prep')

        print(10*'-')


Atualizando campanha: Soluções Digitais
Soluções Digitais já atualizado!
----------
Atualizando campanha: Sustentabilidade
Sustentabilidade já atualizado!
----------
Atualizando campanha: Safrinha
Safrinha já atualizado!
----------
Atualizando campanha: Minhas Finanças
Minhas Finanças já atualizado!
----------
Atualizando campanha: Segurança
Segurança já atualizado!
----------
Atualizando campanha: Cultura
Cultura já atualizado!
----------
Atualizando campanha: Educação Financeira
Educação Financeira já atualizado!
----------
Atualizando campanha: Oportunidades
Atualizando Facebook.
Facebook já atualizado!
Atualizando GoogleAds.
Atualizando GoogleAds [2022-06-01] -> [2022-06-14]
GoogleAds atualizado!
Atualizando Twitter.
Atualizando Twitter [2022-05-25] -> [2022-06-14]
PROCESSANDO
PROCESSANDO
Coletado de 2022-05-25 até 2022-06-14
Twitter atualizado!
Criando base de BM.
Atualizando Google Analytics [BM].
Google Analytics [BM] já atualizado!
Criando base de GA.
----------


In [3]:
campaigns

,ano,campanha,facebook,google,campaign_manager,twitter,ga,nome_parametrizacao,data_inicio,data_fim
0,2022,Estilo,BB | LL | APOIO MC | ESTILO,BB | LL | ED | Estilo,,,184999455,2022 - Estilo,2022-02-10,2022-03-06
1,2022,Universitários,BB | LL | AN | UNIVERSITÁRIOS,BB | LL | AN | Universitários - Flight 1,awareness-negocios_an-universitarios,"AN - Universitários 1 | Always ON | Março,AN -...",210068672,2022 - Universitários,2022-03-17,2022-04-17
2,2022,Dia das Mães,BB | LL | DT | Dia das Mães,BB | LL | DT | Dia das Mães,,"DT | Dia Das Mães | Abril | AQUECIMENTO,DT | D...",255975225,2022 - Campanhas gerais,2022-04-25,2022-05-08


In [4]:
list_campaings = []

for _, campaign in campaigns.iterrows():

    if campaign['campanha'] == 'Dia das Mães':
        is_active = False
        have_bm = False
        have_cm = False

        is_facebook = False
        is_google = False
        is_twitter = False
        is_analytics = False

        year_path = os.path.abspath(os.path.join('..', 'campanhas', str(campaign['ano'])))

        if not os.path.isdir(year_path):
            os.mkdir(year_path)

        campaign_name = campaign['campanha']
        campaign_path = os.path.abspath(os.path.join(year_path, campaign_name.lower()))

        campaign_param_path = os.path.abspath(os.path.join('..', 'parametrização', campaign['nome_parametrizacao'] + '.xlsm'))

        if not os.path.isfile(campaign_param_path):
            campaign_param_path = os.path.abspath(os.path.join('..', 'parametrização', campaign['nome_parametrizacao'] + '.xlsx'))
        
        campaign_start = campaign['data_inicio'].date()
        campaign_end = campaign['data_fim'].date()

        today = datetime.datetime.today().date()
        yesterday = today - datetime.timedelta(days = 1)

        init_date = campaign_start
        end_date = yesterday

        if campaign_start < today <= campaign_end + datetime.timedelta(days = 1):
            is_active = True

        if not os.path.isdir(campaign_path):
            os.mkdir(campaign_path)
            os.mkdir(os.path.join(campaign_path, 'data'))
            os.mkdir(os.path.join(campaign_path, 'source'))
        
        if True:
            param = pd.read_excel(campaign_param_path, sheet_name = 'mas_parametrizacao')
            param.drop(columns = ['id_parametro.1', 'cta', 'term'], inplace = True)
            param.fillna('', inplace = True)

            param.rename(columns = {'Nome do criativo (para programação nas plataformas)': 'ad_name'}, inplace = True)
            
            facebook_accounts = campaign['facebook'].split(',') if campaign['facebook'] else False
            google_accounts = campaign['google'].split(',') if campaign['google'] else False
            twitter_campaigns = campaign['twitter'].split(',') if campaign['twitter'] else False
            cm_campaigns = campaign['campaign_manager'].split(',') if campaign['campaign_manager'] else False

            view_id = str(campaign['ga'])

            if view_id:
                is_analytics = True

            print(f'Atualizando campanha: {campaign_name}')

            concat_pr = []
            concat_ov = []
            concat_ev = []

            if facebook_accounts:
                have_bm = True
                is_facebook = True

                facebook = update_bm(campaign_path, init_date, bm = 'Facebook', accounts = facebook_accounts, bm_function = get_facebook)
                facebook.fillna('', inplace = True)

                no_content_facebook = facebook[facebook['content'] == '']
                content_facebook = facebook.drop(facebook[facebook['content'] == ''].index)

                content_facebook_pr = merge_params(content_facebook, param, campaign_path, 'facebook', content = True)
                no_content_facebook_pr = merge_params(no_content_facebook, param, campaign_path, 'facebook', content = False)

                facebook_pr = pd.concat([content_facebook_pr, no_content_facebook_pr], axis = 0, ignore_index = True)
                concat_pr.append(facebook_pr)

            if google_accounts:
                have_bm = True
                is_google = True

                googleAds = update_bm(campaign_path, init_date, bm = 'GoogleAds', accounts = google_accounts, bm_function = get_googleAds)
                googleAds.fillna('', inplace = True)

                no_content_googleAds = googleAds[googleAds['content'] == '']
                content_googleAds = googleAds.drop(googleAds[googleAds['content'] == ''].index)  

                content_googleAds_pr = merge_params(content_googleAds, param, campaign_path, 'googleAds', content = True)
                no_content_googleAds_pr = merge_params(no_content_googleAds, param, campaign_path, 'googleAds', content = False)

                googleAds_pr = pd.concat([content_googleAds_pr, no_content_googleAds_pr], axis = 0, ignore_index = True)
                concat_pr.append(googleAds_pr)

            if twitter_campaigns:
                have_bm = True
                is_twitter = True

                twitter = update_bm(campaign_path, init_date, bm = 'Twitter', campaigns = twitter_campaigns, bm_function = get_twitter)
                twitter.fillna('', inplace = True)

                no_content_twitter = twitter[twitter['content'] == '']
                content_twitter = twitter.drop(twitter[twitter['content'] == ''].index)   
                
                content_twitter_pr = merge_params(content_twitter, param, campaign_path, 'twitter', content = True)
                no_content_twitter_pr = merge_params(no_content_twitter, param, campaign_path, 'twitter', content = False)
            
                twitter_pr = pd.concat([content_twitter_pr, no_content_twitter_pr], axis = 0, ignore_index = True)
                concat_pr.append(twitter_pr)

            if have_bm and is_analytics:
                bm_campaigns = []
                bm_campaigns += list(googleAds.campaign.str.replace('  ', ' ').unique())
                bm_campaigns += list(facebook.campaign.unique())

                bm_overview, bm_events = update_ga(campaign_path, view_id, init_date, ga = 'bm', campaigns = bm_campaigns, ga_function = get_googleAnalytics_bm)

                if is_facebook:
                    facebook_ov, facebook_ev = merge_ga(facebook_pr, bm_overview, bm_events, bm = 'facebook')
                    concat_ov.append(facebook_ov)
                    concat_ev.append(facebook_ev)
                    
                if is_google:
                    googleAds_ov, googleAds_ev = merge_ga(googleAds_pr, bm_overview, bm_events, bm = 'google')
                    concat_ov.append(googleAds_ov)
                    concat_ev.append(googleAds_ev)

                if is_twitter:
                    twitter_ov, twitter_ev = merge_ga(twitter_pr, bm_overview, bm_events, bm = 'twitter')
                    concat_ov.append(twitter_ov)
                    concat_ev.append(twitter_ev)

            if cm_campaigns:
                have_cm = True

                campaign_manager = update_bm(campaign_path, init_date, bm = 'CampaignManager', campaigns = cm_campaigns, bm_function = get_CampaignManager)
                campaign_manager_pr = merge_cm_params(campaign_manager, param, campaign_path)
                concat_pr.append(campaign_manager_pr)

            if have_cm and is_analytics:
                cm_overview, cm_events = update_ga(campaign_path, view_id, init_date, ga = 'cm', campaigns = cm_campaigns, ga_function = get_googleAnalytics_cm)
                campaign_manager_ov, campaign_manager_ev = merge_ga_cm(campaign_manager_pr, cm_overview, cm_events)
                concat_ov.append(campaign_manager_ov)
                concat_ev.append(campaign_manager_ev)
            
            source_folder = os.path.abspath(os.path.join(campaign_path, 'source'))
            pr = os.path.abspath(os.path.join(source_folder, 'concat_pr.xlsx'))
            gaov = os.path.abspath(os.path.join(source_folder, 'concat_ga.xlsx'))

            concat_pr_all = pd.concat(concat_pr, axis = 0, ignore_index = True)
            concat_pr_all.to_excel(pr, index = False, sheet_name = 'concat_pr')

            
            if is_analytics:
                concat_gaov = pd.concat(concat_ov, axis = 0, ignore_index = True)
                concat_gaev = pd.concat(concat_ev, axis = 0, ignore_index = True)
                writer = pd.ExcelWriter(gaov)
                concat_gaov.to_excel(writer, index = False, sheet_name = 'concat_gaov')
                concat_gaev.to_excel(writer, index = False, sheet_name = 'concat_gaev')
                writer.save()


Atualizando campanha: Dia das Mães
Atualizando Facebook.
Atualizando Facebook [2022-04-25] -> [2022-05-09]
Facebook atualizado!
Atualizando GoogleAds.
Atualizando GoogleAds [2022-04-25] -> [2022-05-09]
GoogleAds atualizado!
Atualizando Twitter.
Atualizando Twitter [2022-04-25] -> [2022-05-09]
Coletado de 2022-04-25 até 2022-05-09
Twitter atualizado!
Atualizando Google Analytics [BM].
Atualizando Google Analytics [BM] [2022-04-25] -> [2022-05-09]
[]
Google Analytics [BM] atualizado!


In [20]:
pr_df = pd.read_excel('C:\\Users\\ricardo.valladares\\Documents\\teste_etl\\campanhas\\2022\\universitários\\data\\facebook_prep.xlsx')
pr_df = pr_df.groupby(['date', 'content', 'source'], as_index = False).sum()

In [34]:
ga_overview = pd.read_excel('C:\\Users\\ricardo.valladares\\Documents\\teste_etl\\campanhas\\2022\\universitários\\data\\bm_googleAnalytics_prep.xlsx', sheet_name = 'ga_overview')
ga_events = pd.read_excel('C:\\Users\\ricardo.valladares\\Documents\\teste_etl\\campanhas\\2022\\universitários\\data\\bm_googleAnalytics_prep.xlsx', sheet_name = 'ga_events')

In [35]:
ov_copy = ga_overview.copy()
ev_copy = ga_events.copy()

overview_bm = ov_copy[ov_copy['source'].isin(['fb', 'ig'])]
events_bm = ev_copy[ev_copy['source'].isin(['fb', 'ig'])]

pr_df['date'] = pd.to_datetime(pr_df['date'])
ov_copy['date'] = pd.to_datetime(ov_copy['date'])
ev_copy['date'] = pd.to_datetime(ev_copy['date'])

In [14]:
bm_metrics = [column for column in pr_df.columns if pr_df[column].dtypes not in ['object', 'datetime64[ns]']]
ga_metrics = [column for column in overview_bm.columns if ((overview_bm[column].dtypes not in ['object', 'datetime64[ns]']) and ('event' not in column))]

In [21]:
pr_df[['cost', 'impressions']].sum()

cost           7.235945e+05
impressions    1.732865e+08
dtype: float64

In [22]:
overview_bm[['sessions', 'totalevents']].sum()

sessions       54225
totalevents    18943
dtype: int64

In [23]:
events_bm[['totalevents']].sum()

totalevents    18943
dtype: int64

In [42]:
overview_bm['ga_id'] = overview_bm['date'].astype(str) + '__' + overview_bm['source'] + '__' + overview_bm['content']
events_bm['ga_id'] = events_bm['date'].astype(str) + '__' + events_bm['source'] + '__' + events_bm['content']

In [24]:
overview_bm.drop(columns = [column for column in (pr_df.columns.intersection(overview_bm.columns)) if column not in ['content', 'source', 'date']], inplace = True)
bm_join_ov = pr_df.merge(overview_bm, on = ['date', 'source', 'content'], how = 'left')

In [25]:
bm_join_ov[['sessions', 'totalevents']].sum()

sessions       52859.0
totalevents    18841.0
dtype: float64

In [26]:
pr_df['check_join'] = ''
overview_bm['check_join'] = ''

pr_df['check_join'] = pr_df['date'].astype(str) + '__' + pr_df['source'] + '__' + pr_df['content']
overview_bm['check_join'] = overview_bm['date'].astype(str) + '__' + overview_bm['source'] + '__' + overview_bm['content']

bm_join_ov['juncao'] = ''
bm_join_ov.loc[~bm_join_ov['sessions'].isnull(), 'juncao'] = 'ok'
bm_join_ov.loc[bm_join_ov['sessions'].isnull(), 'juncao'] = 'ga_ausente'

overview_not_bm = overview_bm[~overview_bm['check_join'].isin(pr_df['check_join'])]
overview_not_bm.drop(columns = [column for column in (pr_df.columns.intersection(overview_not_bm.columns)) if column not in ['source', 'content']], inplace = True)

overview_not_bm = overview_not_bm.merge(pr_df[~pr_df.duplicated(['source', 'content'])], on = ['source', 'content'], how = 'left')
overview_not_bm.loc[:, bm_metrics] = 0

In [63]:
overview_not_bm['juncao'] = ''
overview_not_bm.loc[overview_not_bm['impressions'].isnull(), 'juncao'] = 'erro_ga'
overview_not_bm.loc[~overview_not_bm['impressions'].isnull(), 'juncao'] = 'bm_ausente'

bm_gaoverview = pd.concat([bm_join_ov, overview_not_bm], axis = 0, ignore_index = True)
bm_gaoverview.drop(columns = [column for column in events_bm.columns.intersection(bm_gaoverview.columns) if column not in ['ga_id']], inplace = True)

In [30]:
bm_gaoverview[['impressions', 'cost', 'sessions', 'totalevents']].sum()

impressions    1.732865e+08
cost           7.235945e+05
sessions       5.422500e+04
totalevents    1.894300e+04
dtype: float64

In [64]:
bm_gaevents = events_bm.merge(bm_gaoverview[~bm_gaoverview['ga_id'].isnull()], on = 'ga_id', how = 'left')

bm_gaevents.drop(columns = bm_metrics + ga_metrics, inplace = True)

In [65]:
bm_gaevents[['totalevents']].sum()

totalevents    18943
dtype: int64

In [36]:
events_bm[['totalevents']].sum()

totalevents    18943
dtype: int64

In [66]:
bm_gaevents

,date,campaign,source,medium,content,eventcategory,eventaction,eventlabel,totalevents,uniqueevents,sessionswithevent,ga_id,juncao,check_join
0,2022-03-17,2022_an_univer,fb,video-1200x1200-15s,inst_universitario_br_cpm_id1000220385005,/pbb/pagina-inicial/universitario,clique-menu-principal,Limite de R$ 1.500,1,1,1,2022-03-17__fb__inst_universitario_br_cpm_id10...,ok,NaN
1,2022-03-17,2022_an_univer,fb,video-1200x1200-15s,inst_universitario_br_cpm_id1000220385005,/pbb/pagina-inicial/universitario,clique-menu-principal,Sem pegadinhas,1,1,1,2022-03-17__fb__inst_universitario_br_cpm_id10...,ok,NaN
2,2022-03-17,2022_an_univer,fb,video-1200x1200-15s,inst_universitario_br_cpm_id1000220385005,/pbb/pagina-inicial/universitario,home-beneficios1,limite de r$ 1.500,1,1,0,2022-03-17__fb__inst_universitario_br_cpm_id10...,ok,NaN
3,2022-03-17,2022_an_univer,fb,video-1200x1200-15s,inst_universitario_br_cpm_id1000220385005,/pbb/pagina-inicial/universitario,home-beneficios1,sem pegadinhas,1,1,0,2022-03-17__fb__inst_universitario_br_cpm_id10...,ok,NaN
4,2022-03-17,2022_an_univer,ig,video-1200x1200-15s,inst_universitario_br_cpm_id1000020385000,/pbb/pagina-inicial/universitario,clique-menu-principal,Peça o seu cartão,1,1,0,2022-03-17__ig__inst_universitario_br_cpm_id10...,ok,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4071,2022-04-23,2022_an_univer,ig,video-1200x1200-15s,inst_universitario_br_cpv_id1000020385002,/pbb/pagina-inicial/universitario,clique-menu-principal,Sem pegadinhas,2,1,0,2022-04-23__ig__inst_universitario_br_cpv_id10...,bm_ausente,2022-03-19__ig__inst_universitario_br_cpv_id10...
4072,2022-04-23,2022_an_univer,ig,video-1200x1200-15s,inst_universitario_br_cpv_id1000020385002,/pbb/pagina-inicial/universitario,home-beneficios1,sem pegadinhas,2,1,0,2022-04-23__ig__inst_universitario_br_cpv_id10...,bm_ausente,2022-03-19__ig__inst_universitario_br_cpv_id10...
4073,2022-04-26,2022_an_univer,ig,video-1200x1200-15s,inst_universitario_br_cpm_id1003320385068,/pbb/pagina-inicial/universitario,clique-banner-superior,https://www.bb.com.br/docs/portal/dimac/Banner...,2,1,1,2022-04-26__ig__inst_universitario_br_cpm_id10...,bm_ausente,2022-04-01__ig__inst_universitario_br_cpm_id10...
4074,2022-04-28,2022_an_univer,ig,video-1200x1200-15s,inst_universitario_br_cpm_id1003320385068,/pbb/pagina-inicial/universitario,clique-menu-principal,Peça o seu cartão,1,1,1,2022-04-28__ig__inst_universitario_br_cpm_id10...,bm_ausente,2022-04-01__ig__inst_universitario_br_cpm_id10...


In [53]:
events_bm.shape

(4076, 13)

In [61]:
list(bm_gaevents.ga_id)

['2022-03-17__fb__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__fb__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__fb__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__fb__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000020385000',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000020385000',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000220385005',
 '2022-03-17__ig__inst_universitario_br_cpm_id1000220385005',
 '2022-0